In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/shubhanagrawal/End-to-End-Chest-Cancer-Classification-using-MLfLow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="shubhanagrawal"
os.environ["MLFLOW_TRACKING_PASSWORD"]="fabfa866dd1e0c627bed32f7f075a46397e47e71"



In [2]:
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/shubhanagrawal/End-to-End-Chest-Cancer-Classification-using-MLfLow-DVC.mlflow")


In [3]:
import dagshub
dagshub.init(repo_owner='shubhanagrawal', repo_name='End-to-End-Chest-Cancer-Classification-using-MLfLow-DVC', mlflow=True)


Accessing as shubhanagrawal

Initialized MLflow to track repo "shubhanagrawal/End-to-End-Chest-Cancer-Classification-using-MLfLow-DVC"

Repository shubhanagrawal/End-to-End-Chest-Cancer-Classification-using-MLfLow-DVC initialized!

In [4]:
import os

In [5]:
%pwd


'd:\\Projects\\End-to-End-Chest-Cancer-Classification-using-MLfLow-DVC\\research'

In [7]:

import tensorflow as tf


In [6]:
os.chdir("../")

In [8]:

model = tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/entbappy/chest-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [12]:

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [13]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
import numpy as np
import json
import mlflow
from urllib.parse import urlparse
from pathlib import Path
import tensorflow as tf

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()

        # Evaluate model loss & accuracy
        self.score = self.model.evaluate(self.valid_generator)

        # Get predictions for additional metrics
        y_true = self.valid_generator.classes
        y_pred_probs = self.model.predict(self.valid_generator)
        y_pred = np.argmax(y_pred_probs, axis=1)

        # Compute metrics
        self.precision = precision_score(y_true, y_pred, average="weighted")
        self.recall = recall_score(y_true, y_pred, average="weighted")
        self.f1 = f1_score(y_true, y_pred, average="weighted")
        self.conf_matrix = confusion_matrix(y_true, y_pred).tolist()

        self.save_score()

    def save_score(self):
        scores = {
            "loss": self.score[0],
            "accuracy": self.score[1],
            "precision": self.precision,
            "recall": self.recall,
            "f1_score": self.f1,
            "confusion_matrix": self.conf_matrix
        }
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1],
                "precision": self.precision,
                "recall": self.recall,
                "f1_score": self.f1
            })

            # Do NOT use model registry if using DagsHub
            if "dagshub" in self.config.mlflow_uri.lower():
                mlflow.keras.log_model(self.model, "model")  # Without registry
            elif tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-08-24 17:14:33,108: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-24 17:14:33,116: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-24 17:14:33,118: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
4/4 [==============================] - 10s 2s/step
[2025-08-24 17:14:50,677: INFO: common: json file saved at: scores.json]


MlflowException: API request to endpoint /api/2.0/mlflow/runs/create failed with error code 404 != 200. Response body: ''